### 1. Import the essential libraries

In [22]:
import os
import sys
import cv2
import random
import joblib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

from skimage.color import rgb2lab, lab2rgb
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

from tqdm import tqdm_notebook

### 1.1 Define some helpful functions

In [23]:
def describeImage(Xsub_rgb,labels=["R","G","B"]):
    print("______________________________")
    for i, lab in enumerate(labels):
        mi = np.min(Xsub_rgb[:,:,i])
        ma = np.max(Xsub_rgb[:,:,i])
        print("{} : MIN={:8.4f}, MAX={:8.4f}".format(lab,mi,ma))

In [24]:
def extract_single_dim_from_LAB_convert_to_RGB(image,idim):
    '''
    image is a single lab image of shape (None,None,3)
    '''
    z = np.zeros(image.shape)
    if idim != 0 :
        z[:,:,0]=80 ## I need brightness to plot the image along 1st or 2nd axis
    z[:,:,idim] = image[:,:,idim]
    z = lab2rgb(z)
    return(z)

### 2. Set the paths for *train* and *test* data

In [25]:
train_images_path = os.path.join("/media/andretri7/WD/Datasets/flower_images/flower_images")

### 3. Select n=19 images

In [26]:
train_images = np.array(os.listdir(train_images_path))
# train_images = train_images[np.random.choice(train_images.shape[0], size=100, replace=False)]

### 4. Create the Palette
* ####  For Every Image:
  * #### 4.1. Convert it to LAB Color Space
  * #### 4.2. Extract the Colors
  * #### 4.3. Perform the Color Space Segmentation using Mini-Batch k-Means @ 32 Colors

In [27]:
colorPalette = MiniBatchKMeans(n_clusters=32, random_state=100, compute_labels=True)

for img in tqdm_notebook(train_images):
    # 1: Read Image
    imgRGB = cv2.imread(os.path.join(train_images_path, img))
    imgRGB = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2RGB)
    # 2: Normalize Values from [0, 255] to [0, 1]
    imgRGB = imgRGB/255
    # 3: Convert Image from RGB to LAB
    imgLAB = rgb2lab(imgRGB)
    # 4: Get (a, b) Coordinates from Image and Append to Color Palette
    imgLAB_colors = np.vstack(imgLAB[:, :, 1:3])
    colorPalette.partial_fit(imgLAB_colors)

# CHECKPOINT
* ### Save the Color Palette and the Training Image Data for the following notebooks

In [28]:
if not os.path.exists(os.path.join('.', 'res_all')):
    os.makedirs(os.path.join('.', 'res_all'))
    
save_path = os.path.join('.', 'res_all')

In [29]:
joblib.dump(colorPalette, os.path.join(save_path,'colorPalette.joblib'))
joblib.dump(train_images, os.path.join(save_path,'trainImages.joblib'))

['./res_all/trainImages.joblib']